##  The Grover Algorithm: Implementation


_course: quantum cryptography for beginners
<br>date: 30 november 2022
<br>author: burton rosenberg_



### The  Walsh Hadamard Transformation

To implement the Grover Algorithm we need to implement two refections. The oracle reflection:

$$
U_f:\;|\phi\rangle \longrightarrow \sum_i (-1)^{(i==j)}\,|i\rangle
$$

and the flip around the average,

$$
    U_{r'} = 2 |h_0\rangle\langle h_0| - I
$$

Where $h_0$ is the result of putting zero through a bank of Hadamard transfomations,

$$
  h_0 = H^{n} |0\rangle
$$

and in general the Walsh-Hadamard basis is,

$$
 h_i = H^{n} |i\rangle
$$

whichi is a basis as the transformation is an isometry and the i's are a basis. The vectors in this basis are remarkably a pattern of +1's and -1's that described as,

$$
H^{n} |i\rangle = \sum_j (-1)^{i\cdot j}\,|i\rangle
$$

where you are to interpret $i\cdot j$ as the inner product in $\mathcal{Z}_2{}^n$ having expressed $i, j\in \mathcal{Z}_2{}^n$ by looking at their binary expansion. For instance,


|  i | +/-j |
| ---- | ---- |
| 0 | + + + + |
| 1 | + - + - |
| 2 | + + - - |
| 3 | + - - + |

In fact, we do not implement the reflection around $h_0$ rather around it perpendicular in the plane containing it and the vector $|j\rangle$ representing the unknown. This upsets our plan by a minus sign, as the two reflections differ by a half rotation $x\rightarrow -x$,

\begin{eqnarray}
U_r &=& -U_{r'} \\
&=& I - 2 |h_0\rangle\langle h_0|  \\
&=& \sum_i |h_i\rangle \langle h_i | - 2 |h_0\rangle\langle h_0| \\
&=& \sum_i (-1)^{i==0}\, |h_i\rangle \langle h_i | 
\end{eqnarray}

The final element is how to reflect around $|i\rangle$, expressed as,

$$
U_i  = \sum_j (-1)^{i==j}\,\,|j\rangle
$$

This is done with a controlled Z gate, applied to any qubit and controlled by all the remaining qubits. 

To reflect around $|0\rangle$, negate with an X all but one of the qubits, use Tofolli gates and ancillas as needed to logically and (classical) all the qubits, and use the result as the control to a controlled Z on the uninverted qubit. Then uncompute the anciliaries and the X negations.

##### Example

In [1]:
# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
import numpy as np
import math
import cmath
np.set_printoptions(precision=2,floatmode='fixed',suppress=True)

#Aer.backends()

def Ur():
    qc = QuantumCircuit(4)
    for i in range(3):
        qc.h(i)
        qc.x(i)
    qc.ccx(0,1,3)
    qc.cz(3,2)
    qc.ccx(0,1,3)
    for i in range(3):
        qc.x(i)
        qc.h(i)
    return qc

print(Ur())

     ┌───┐┌───┐             ┌───┐┌───┐
q_0: ┤ H ├┤ X ├──■───────■──┤ X ├┤ H ├
     ├───┤├───┤  │       │  ├───┤├───┤
q_1: ┤ H ├┤ X ├──■───────■──┤ X ├┤ H ├
     ├───┤├───┤  │       │  ├───┤├───┤
q_2: ┤ H ├┤ X ├──┼───■───┼──┤ X ├┤ H ├
     └───┘└───┘┌─┴─┐ │ ┌─┴─┐└───┘└───┘
q_3: ──────────┤ X ├─■─┤ X ├──────────
               └───┘   └───┘          



### The Grover's Algorithm on a GPU

It might be good to refelction on what is the quantum advantage of Grover's algorithm. To do so we can implement it non-quantum. The Walsh-Hadamard transform can be done on a GPU with log n time complexity. Since the fan-in to the controled Z will require a log n tree of ands, the time complexity will match that of the quantum algorithm.

However, we will need an exponential number of GPU threads. While $n$ qubits can encode $2^n$ complex numbers, we will need $2^n$ GPU threads, one for each complex number. However, there are about 100,000 GPU threads on some Nvidia cards. It might be the bottle neck of memory access with is truely the problem with Grovers done on a classical computer.

See the wikipedia description of the [Fast Walsh Hadamard](https://en.wikipedia.org/wiki/Fast_Walsh%E2%80%93Hadamard_transform).


In [2]:

import numpy as np
import math

class Grover_GPU:
    
    def __init__(self,k):
        self.iter = int(math.sqrt(2**k))
        self.phi = np.ones(k)/math.sqrt(k)
        pass
    
    @staticmethod
    def wh_transform(a):
        # assume len(a) = 2^i
        
        def wh_kernel(a,i,d):
            if i&d==0: a[i], a[i+d] = a[i]+a[i+d], a[i]-a[i+d]
            return 

        n = len(a)    
        d = n//2
        while True:
            # parrallel kernel launches
            for i in range(n): wh_kernel(a,i,d)
            if d==1: break
            d = d//2

    def reflect_h0(self):
        self.wh_transform(self.phi)
        self.phi[0] = -self.phi[0]
        self.wh_transform(self.phi)
        self.phi = self.phi/len(self.phi)
        return

    def reflect_j(self,j):
        self.phi[j] = -self.phi[j]
        return

    def grover_step(self,j):
        self.reflect_j(j)
        self.reflect_h0()
        return
    
    def observation(self,j):
        d = self.phi[j]
        return d*d

    def coords(self,j):
        r = self.phi[j]*self.phi[j]
        if r>1.0:
            r = 1.0
        return (math.sqrt(r),math.sqrt(1-r))

g = Grover_GPU(128)
j = 4
for i in range(16):
    g.grover_step(j)
    print(f'obs: {g.observation(j):.4f}')
    

obs: 0.0689
obs: 0.1834
obs: 0.3372
obs: 0.5111
obs: 0.6837
obs: 0.8335
obs: 0.9420
obs: 0.9956
obs: 0.9878
obs: 0.9194
obs: 0.7991
obs: 0.6416
obs: 0.4666
obs: 0.2957
obs: 0.1502
obs: 0.0480


### Exercise A

Test the reflection circuit `Ur_3`. The following code places that code into a `Grover_Circuit` class that we will add to later. It prefixes it with a qiskit call that initializes an arbitrary statevector. This is not the same as initializing individual qubits, because for the three qubits there are 8 numbers to supply. 

The state is then acted upon and `get_statevector` gets the actual result, not a measurement. 

Your work is all in the `BasisSpan` class, in the calculation of $h_o{}^\perp$ and the method `at_theta_perp`. The class has two other important vectors, $h_o$, the vector of all 1's (scaled to unit length) and $u_j$, the vector that is 0 in all indices except j, where it is 1. 

Hint: $h_o{}^\perp$ satisfies, 

$$
h_o{}^\perp \cdot h_o = (\alpha\, h_o +\beta\, u_j) \cdot h_o = 0.
$$

The code `at_theta_perp` returns,

$$
(\cos\theta)\,h_o^\perp + (\sin\theta)\, u_j
$$

The test routine makes several vectors at various $\theta$ and compares the result to the vectors at $-\theta$.


In [57]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from numpy import sqrt
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator

# https://qiskit.org/textbook/ch-states/representing-qubit-states.html


# the class is done, it is a stub version of the Grover Circuit 
# containing just a reflection for 3 qubits, and a test widget

class Grover_Circuit:
        
    def __init__(self):
        self.n = 3
        self.sim = Aer.get_backend('aer_simulator')
        self.qc = None
        pass

        pass
    
    def Ur_3(self,qc):
        for i in range(3):
            qc.h(i)
            qc.x(i)
        qc.ccx(0,1,3)
        qc.cz(3,2)
        qc.ccx(0,1,3)
        for i in range(3):
            qc.x(i)
            qc.h(i)
        qc.save_statevector()
        return qc
    
    def ur3_initialized_circuit(self,init_state):
        qc = QuantumCircuit(4)
        qc.initialize(init_state,[0,1,2])
        return self.Ur_3(qc)


# this test def is done. it tests the circuit

def test(n,j,step,print_state=False,max_iters=100):

    sim = Aer.get_backend('aer_simulator')
    bs = BasisSpan(j,n)
    grover = Grover_Circuit()

    pi_step = math.pi/step

    correct = True
    for i in range(max_iters):
        
        # build the test state and predicted result
        test_state = bs.at_theta_perp(i*pi_step)
        predicted_state = bs.at_theta_perp(-i*pi_step)
        
        # run the circuit on the test state to get the test state reflected
        qc = grover.ur3_initialized_circuit(test_state)
        result = sim.run(qc).result()
        test_state_reflected = result.get_statevector()
        
        
        # report result
        res = np.allclose(predicted_state,test_state_reflected.data[:n])
        correct = correct and res
        print(f'{i*pi_step:.4f}: {res}',end='')
        if print_state:
            print(f', {predicted_state}')
        else:
            print()
            
        # break of test was along the h_o axis
        if np.allclose(test_state,bs.h_o):
            break

    return res

   
# this class you need to do. the idea of grovers is we work in a 2-d real
# span of h_o, the all 1's vector, and u_j, the standard unit vector for index j

# for reasons of whats most practicaly, the grover implementation actually reflects
# aro|und the perpendicular to h_o in the h_o/u_j plane.


class BasisSpan:
    
    def __init__(self,j,n):
        self.n = n
        self.j = j
        self.h_o = np.ones(n)/math.sqrt(n)
        self.u_j = np.zeros(n)
        self.u_j[j] = 1.0
        
        # calculate the perpendicular to h_o in the plane
        
        self.h_o_perp = np.zeros(self.n) # do this
        
    def at_theta_perp(self,theta):
        # create a vector that is theta radians from the h_o_perp vector
        # in the h_o/u_j plane
        
        phi = np.zeros(self.n) # do this
        return phi

try:
    t = test(8,0,32,print_state=True) 
except Exception as e:
    print(f'Sorry, it did not work, error is |{e}|')
    t = False

if t:
    print('*** passed ***')
else:
    print('*** failed ***')

Sorry, it did not work, error is |'Sum of amplitudes-squared does not equal one.'|
*** failed ***


### Exercise B

Complete the 3 qubit Grovers circuit and test.


In [76]:
class Grover_Circuit:
        
    def __init__(self):
        self.n = 3
        self.sim = Aer.get_backend('aer_simulator')
        self.qc = None
        pass
    
    def Ur_3(self,qc):
        for i in range(3):
            qc.h(i)
            qc.x(i)
        qc.ccx(0,1,3)
        qc.cz(3,2)
        qc.ccx(0,1,3)
        for i in range(3):
            qc.x(i)
            qc.h(i)
        qc.save_statevector()
        return qc
    
    def ccz(self,qc):
        # a controlled z. controls on 0 and 1, output on 2, ancilla qubit is 3
        #
        # code below
        
        # code above
        return qc
    
    def select_j(self,qc,j):
        # create a bank of X operators on qubits so that j maps to 1
        #
        # code below
        
        #code above
        return qc

    def initialize_h_o(self):
        qc = QuantumCircuit(4)
        for i in range(3):
            qc.h(i)
        return qc

    def grover_one_phase(self,j):
        qc = self.initialize_h_o()
        self.select_j(qc,j)
        self.ccz(qc)
        self.select_j(qc,j)
        self.Ur_3(qc)
        qc.measure_all()
        self.qc = qc
    
    def run(self,print_circuit=False):
        assert self.qc
        if print_circuit:
            print(self.qc.draw())
        result = self.sim.run(self.qc).result()
        counts = result.get_counts()
        x = { k: counts[k] for k in sorted(counts,key=counts.get,reverse=True)}
        print(x)
     
    def ur3_initialized_circuit(self,init_state):
        qc = QuantumCircuit(4)
        qc.initialize(init_state,[0,1,2])
        return self.Ur_3(qc)

j = 4
gc = Grover_Circuit()
for j in range(8):
    gc.grover_one_phase(j)
    gc.run()

{'0011': 156, '0110': 144, '0111': 132, '0001': 127, '0101': 119, '0000': 117, '0010': 115, '0100': 114}
{'0111': 135, '0100': 134, '0011': 132, '0010': 130, '0000': 127, '0101': 127, '0110': 121, '0001': 118}
{'0010': 139, '0111': 138, '0101': 136, '0011': 131, '0000': 129, '0100': 126, '0110': 119, '0001': 106}
{'0011': 154, '0001': 145, '0100': 137, '0010': 127, '0000': 122, '0110': 119, '0111': 114, '0101': 106}
{'0100': 151, '0001': 138, '0111': 137, '0000': 131, '0110': 130, '0010': 117, '0101': 111, '0011': 109}
{'0101': 149, '0010': 143, '0011': 138, '0111': 128, '0110': 118, '0000': 118, '0100': 117, '0001': 113}
{'0101': 146, '0100': 143, '0001': 137, '0110': 136, '0010': 127, '0111': 116, '0011': 115, '0000': 104}
{'0011': 148, '0110': 146, '0111': 130, '0101': 127, '0001': 125, '0000': 117, '0010': 117, '0100': 114}


### Exercise C

Create a python function that builds a k qubit Grovers circuit.

---------

# Answers

-----------

### Answer A

In [77]:

# go back and run the question cell, then this answer cell

class BasisSpan:
    
    def __init__(self,j,n):
        self.n = n
        self.j = j
        self.h_o = np.ones(n)/math.sqrt(n)
        self.u_j = np.zeros(n)
        self.u_j[j] = 1.0
        h_o_perp = self.h_o - math.sqrt(n)*self.u_j
        self.h_o_perp = h_o_perp /math.sqrt(np.dot(h_o_perp,h_o_perp))
        
    def at_theta_perp(self,theta):
        return math.cos(theta)*self.h_o_perp + math.sin(theta)*self.h_o

try:
    t = test(8,0,32,print_state=True) 
except Exception as e:
    print(f'Sorry, it did not work, error is |{e}|')
    t = False

if t:
    print('*** passed ***')
else:
    print('*** failed ***')

0.0000: True, [-0.94  0.13  0.13  0.13  0.13  0.13  0.13  0.13]
0.0982: True, [-0.97  0.10  0.10  0.10  0.10  0.10  0.10  0.10]
0.1963: True, [-0.99  0.06  0.06  0.06  0.06  0.06  0.06  0.06]
0.2945: True, [-1.00  0.03  0.03  0.03  0.03  0.03  0.03  0.03]
0.3927: True, [-1.00 -0.01 -0.01 -0.01 -0.01 -0.01 -0.01 -0.01]
0.4909: True, [-0.99 -0.05 -0.05 -0.05 -0.05 -0.05 -0.05 -0.05]
0.5890: True, [-0.97 -0.09 -0.09 -0.09 -0.09 -0.09 -0.09 -0.09]
0.6872: True, [-0.95 -0.12 -0.12 -0.12 -0.12 -0.12 -0.12 -0.12]
0.7854: True, [-0.91 -0.16 -0.16 -0.16 -0.16 -0.16 -0.16 -0.16]
0.8836: True, [-0.87 -0.19 -0.19 -0.19 -0.19 -0.19 -0.19 -0.19]
0.9817: True, [-0.81 -0.22 -0.22 -0.22 -0.22 -0.22 -0.22 -0.22]
1.0799: True, [-0.75 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25]
1.1781: True, [-0.68 -0.28 -0.28 -0.28 -0.28 -0.28 -0.28 -0.28]
1.2763: True, [-0.61 -0.30 -0.30 -0.30 -0.30 -0.30 -0.30 -0.30]
1.3744: True, [-0.53 -0.32 -0.32 -0.32 -0.32 -0.32 -0.32 -0.32]
1.4726: True, [-0.44 -0.34 -0.34 -0.34 -

### Answer B

In [78]:
class Grover_Circuit:
        
    def __init__(self):
        self.n = 3
        self.sim = Aer.get_backend('aer_simulator')
        self.qc = None
        pass
    
    def Ur_3(self,qc):
        for i in range(3):
            qc.h(i)
            qc.x(i)
        qc.ccx(0,1,3)
        qc.cz(3,2)
        qc.ccx(0,1,3)
        for i in range(3):
            qc.x(i)
            qc.h(i)
        qc.save_statevector()
        return qc
    
    def ccz(self,qc):
        # a controlled z. controls on 0 and 1, output on 2, ancilla qubit is 3
        qc.ccx(0,1,3)
        qc.cz(2,3)
        qc.ccx(0,1,3)
        return qc
    
    def select_j(self,qc,j):
        # create a bank of X operators on qubits so that j maps to 1
        for i in range(self.n):
            if j%2==0:
                qc.x(i)
            j = j//2
        return qc

    def initialize_h_o(self):
        qc = QuantumCircuit(4)
        for i in range(3):
            qc.h(i)
        return qc

    def grover_one_phase(self,j):
        qc = self.initialize_h_o()
        self.select_j(qc,j)
        self.ccz(qc)
        self.select_j(qc,j)
        self.Ur_3(qc)
        qc.measure_all()
        self.qc = qc
    
    def run(self,print_circuit=False):
        assert self.qc
        if print_circuit:
            print(self.qc.draw())
        result = self.sim.run(self.qc).result()
        counts = result.get_counts()
        x = { k: counts[k] for k in sorted(counts,key=counts.get,reverse=True)}
        print(x)
     
    def ur3_initialized_circuit(self,init_state):
        qc = QuantumCircuit(4)
        qc.initialize(init_state,[0,1,2])
        return self.Ur_3(qc)

j = 4
gc = Grover_Circuit()
for j in range(8):
    gc.grover_one_phase(j)
    gc.run()


{'0000': 799, '0110': 45, '0100': 35, '0111': 31, '0001': 31, '0010': 31, '0101': 28, '0011': 24}
{'0001': 821, '0110': 40, '0010': 31, '0000': 29, '0100': 29, '0111': 26, '0101': 25, '0011': 23}
{'0010': 809, '0000': 44, '0110': 36, '0011': 31, '0111': 27, '0001': 27, '0101': 27, '0100': 23}
{'0011': 763, '0001': 47, '0100': 39, '0111': 37, '0000': 35, '0010': 35, '0110': 34, '0101': 34}
{'0100': 763, '0111': 46, '0010': 46, '0110': 40, '0001': 38, '0101': 32, '0000': 30, '0011': 29}
{'0101': 810, '0010': 39, '0111': 36, '0000': 32, '0011': 31, '0110': 27, '0100': 26, '0001': 23}
{'0110': 809, '0100': 40, '0111': 35, '0000': 33, '0010': 31, '0101': 31, '0001': 23, '0011': 22}
{'0111': 794, '0010': 38, '0100': 36, '0001': 36, '0101': 33, '0110': 31, '0011': 31, '0000': 25}


## Answers C

In [79]:
# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
import numpy as np
import math
import cmath
np.set_printoptions(precision=2,floatmode='fixed',suppress=True)

class Grover:

    def __init__(self,n,s,barriers=False):
        assert n>3
        assert s<2**n
        
        self.n = n
        self.k = n - 2  # number of anciliaries needed
        self.qc = QuantumCircuit(n+self.k)
        
        self.hadamard()
        if barriers: 
            self.qc.barrier()
        self.reflect_around_j(s)
        if barriers:
            self.qc.barrier()
        self.reflect_around_plus()
        self.qc.measure_all()
        
    def cnz(self):
        assert self.n>=3
        a = self.n
        self.qc.ccx(0,1,self.n)
        for i in range(2,self.n-1):
            self.qc.ccx(i,a,a+1)
            a += 1
        self.qc.cz(self.n-1,a)
        for i in range(self.n-2,1,-1):
            a -= 1
            self.qc.ccx(i,a,a+1)
        self.qc.ccx(0,1,self.n)
        return

    def reflect_around_plus(self):
        for i in range(self.n):
            self.qc.h(i)
            self.qc.x(i)
        self.cnz()
        for i in range(n):
            self.qc.x(i)
            self.qc.h(i)
        return

    def reflect_around_j(self,j):
        self.select_j(j)
        self.cnz()
        self.select_j(j)
        return

    def select_j(self,j):
        for i in range(self.n):
            if j%2==0:
                self.qc.x(i)
            j = j//2
        return

    def hadamard(self):
        for i in range(self.n):
            self.qc.h(i)
        return
    
    def draw(self):
        print(self.qc.draw())
        
    def run(self,how_many=12):
        simulator = Aer.get_backend('aer_simulator')
        qc = transpile(self.qc,simulator)
        result = simulator.run(qc).result()
        counts = result.get_counts(qc)
        x = { k: counts[k] for k in sorted(counts,key=counts.get,reverse=True)}
        print(x)
                

n = 6 # qubits
s = 4 # secret number

grover = Grover(n,s,barriers=True)
grover.draw()
grover.run()


         ┌───┐ ░ ┌───┐                                                     »
    q_0: ┤ H ├─░─┤ X ├──■───────────────────────────────────────────────■──»
         ├───┤ ░ ├───┤  │                                               │  »
    q_1: ┤ H ├─░─┤ X ├──■───────────────────────────────────────────────■──»
         ├───┤ ░ └───┘  │                                               │  »
    q_2: ┤ H ├─░────────┼────■────────────────────────────────■─────────┼──»
         ├───┤ ░ ┌───┐  │    │                                │  ┌───┐  │  »
    q_3: ┤ H ├─░─┤ X ├──┼────┼────■──────────────────────■────┼──┤ X ├──┼──»
         ├───┤ ░ ├───┤  │    │    │               ┌───┐  │    │  └───┘  │  »
    q_4: ┤ H ├─░─┤ X ├──┼────┼────┼────■───────■──┤ X ├──┼────┼─────────┼──»
         ├───┤ ░ ├───┤  │    │    │    │       │  ├───┤  │    │         │  »
    q_5: ┤ H ├─░─┤ X ├──┼────┼────┼────┼───■───┼──┤ X ├──┼────┼─────────┼──»
         └───┘ ░ └───┘┌─┴─┐  │    │    │   │   │  └───┘  │    │       ┌─┴─┐»